In [1]:
import os

if "src" not in os.listdir():
    os.chdir("../")

In [2]:
import json
import requests

import yaml

In [3]:
with open('conf.yaml') as fh:
    read_data = yaml.load(fh, Loader=yaml.FullLoader)

In [4]:
read_data.keys()

dict_keys(['URL_DB_QDRANT', 'API_DB_QDRANT'])

In [5]:
# url = "https://api.minimaxi.chat/v1/text/chatcompletion_v2"
# API_KEY = read_data["api_minmax"]
# payload = json.dumps({
#     "model":"MiniMax-Text-01",
#     "messages":[
#         {
#             "role":"system",
#             "content":"Ты профессиональный юрист, специалист по российскому праву. Ничего не придумывай. В конце ответа пиши краткое резюме своего ответа, где указывай прямой и сжатый ответ на впрос пользователя."
#         },
#         {
#             "role":"user",
#             "content":"Что делать если не берут а Армию? Я очень хочу служить России."
#         }
#     ],
#     "stream":False,
#     "max_tokens":10000,
#     "temperature":0.9,
#     "top_p":1
# })
# headers = {
#     'Authorization': f'Bearer {API_KEY}',
#     'Content-Type': 'application/json'
# }
# response = requests.request("POST", url, headers=headers, data=payload)
# print(response.json())

In [6]:
url = "https://demo5-fundres.dev.mts.ai/v1/chat/completions"
API_KEY = read_data["api_mts"]
payload = json.dumps({
    "model":"cotype_pro_16k_1.1",
    "messages":[
        {
            "role":"system",
            "content":"Ты модель AI. Пиши коротко."
        },
        {
            "role":"user",
            "content":"Кто ты?"
        }
    ],
})
headers = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}
response = requests.request("POST", url, headers=headers, data=payload)
print(response.json())

KeyError: 'api_mts'

In [1]:
from langchain_community.chat_models.yandex import ChatYandexGPT

In [6]:
import yandex.cloud.ai.foundation_models

ModuleNotFoundError: No module named 'yandex.cloud.ai.foundation_models'

In [11]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True, device="cpu") # Setting use_fp16 to True speeds up computation with a slight performance degradation

In [15]:
score = reranker.compute_score([['query', 'passage passage'*10] for _ in range(200)])
print(score) # -5.65234375

Compute Scores: 100%|██████████| 1/1 [01:31<00:00, 91.04s/it]

[-5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658, -5.822876453399658,

In [7]:
from langchain_openai.chat_models.base import ChatOpenAI

In [8]:
import os

os.environ["OPENAI_API_KEY"] = API_KEY

In [9]:
client = ChatOpenAI(model_name="cotype_pro_16k_1.1", base_url=url)

In [ ]:
client

In [ ]:
client.generate(["text"])

In [25]:
from typing import Any, List, Optional, Dict, Union
from langchain.llms.base import BaseLLM
from langchain.schema import Generation, LLMResult
import requests

class CustomLLM(BaseLLM):
    api_url: str  # URL вашего API
    api_key: str  # API ключ, если требуется
    model_name: str = "cotype_pro_16k_1.1"  # Имя модели

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        # Системное сообщение
        system_message = {
            "role": "system",
            "content": "Ты ассистент, который помогает людям. Пиши на русском языке!"
        }
        # Пользовательское сообщение
        user_message = {
            "role": "user",
            "content": prompt
        }
        payload = {
            "model": self.model_name,
            "messages": [system_message, user_message],
        }
        # Отправка запроса к API
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        response = requests.post(self.api_url, headers=headers, json=payload)
        response.raise_for_status()
        result = response.json()
        
        # Извлечение ответа из результатов
        # Предполагается, что ответ находится в result["choices"][0]["message"]["content"]
        assistant_response = result["choices"][0]["message"]["content"]
        return assistant_response

    def _generate(self, prompts: List[str], stop: Optional[List[str]] = None) -> LLMResult:
        generations = []
        for prompt in prompts:
            text = self._call(prompt, stop)
            generations.append([Generation(text=text)])
        return LLMResult(generations=generations)

    @property
    def _llm_type(self) -> str:
        return "custom_llm"

In [26]:
custom_chat_api = CustomLLM(
    api_url="https://demo5-fundres.dev.mts.ai/v1/chat/completions",
    model_name="cotype_pro_16k_1.1",
    api_key=API_KEY
)

In [ ]:
custom_chat_api.invoke("кто ты?")